### Import libraries and create DataFrame from csv 

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Function to read, preprocess, and aggregate full season data
def read_data():
    # Read the full season data
    data = pd.read_excel("TE_Data_2024.xlsx")

    # Preprocess full season data, drop irrelevant columns and convert data types
    data.drop(['FL', 'ROST'], axis=1, inplace=True)
    convert = data.select_dtypes('object').columns.difference(['Player'])
    data[convert] = data[convert].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce')).fillna(0)
    data['Rank'] = data['Rank'].astype('Int64')

    return data

# Load the full season data
df = read_data()

# Recalculate 'Rank' based on 'FPTS' in descending order
df['Rank'] = df['FPTS'].rank(ascending=False, method='min').astype('int')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')
df.head(20)

,Rank,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,TD.1,G,FPTS,FPTS/G
0,1,Dallas Goedert (PHI),17,20,239,14.1,61,5,0,0,0,0,3,40.9,13.6
1,2,Brock Bowers (LV),18,21,197,10.9,27,4,0,0,0,0,3,37.7,12.6
2,3,Cole Kmet (CHI),15,17,128,8.5,19,0,1,0,0,0,3,33.8,11.3
3,4,Trey McBride (ARI),14,21,122,8.7,21,1,0,0,0,0,3,32.2,10.7
4,5,Isaiah Likely (BAL),12,16,141,11.8,49,1,1,0,0,0,3,32.1,10.7
5,6,Jake Ferguson (DAL),16,23,159,9.9,26,2,0,0,0,0,3,31.9,10.6
6,7,Mike Gesicki (CIN),14,18,156,11.1,37,1,0,0,0,0,3,29.6,9.9
7,8,George Kittle (SF),11,13,116,10.5,26,2,1,0,0,0,2,28.6,14.3
8,9,Hunter Henry (NE),12,18,136,11.3,35,1,0,0,0,0,3,25.6,8.5
9,10,Zach Ertz (WAS),12,13,128,10.7,21,2,0,0,0,0,3,24.8,8.3


### Convert relevant stats to a per-game basis and create the final stats for analysis

In [25]:
# Define columns for per-game calculation
per_game = list(df.columns[2:])

# Define columns to exclude from the per-game calculation
exclude_per_game = ['Y/R', 'LG', 'G', 'FPTS', 'FPTS/G']

# Create columns and convert stats to a per-game basis for the defined columns, not those in the exclude list
for col in per_game:
    if col not in exclude_per_game:
        df[col + '/game'] = (df[col] / df['G']).round(1)

# Define final columns to be used for analysis, combining excluded and new per-game columns
final_columns = exclude_per_game + [col + '/game' for col in per_game if col not in exclude_per_game]

# Display final columns with 'Rank' and 'Player'
df[['Rank', 'Player'] + final_columns].head(10)

,Rank,Player,Y/R,LG,G,FPTS,FPTS/G,REC/game,TGT/game,YDS/game,20+/game,TD/game,ATT/game,YDS.1/game,TD.1/game
0,1,Dallas Goedert (PHI),14.1,61,3,40.9,13.6,5.7,6.7,79.7,1.7,0.0,0.0,0.0,0.0
1,2,Brock Bowers (LV),10.9,27,3,37.7,12.6,6.0,7.0,65.7,1.3,0.0,0.0,0.0,0.0
2,3,Cole Kmet (CHI),8.5,19,3,33.8,11.3,5.0,5.7,42.7,0.0,0.3,0.0,0.0,0.0
3,4,Trey McBride (ARI),8.7,21,3,32.2,10.7,4.7,7.0,40.7,0.3,0.0,0.0,0.0,0.0
4,5,Isaiah Likely (BAL),11.8,49,3,32.1,10.7,4.0,5.3,47.0,0.3,0.3,0.0,0.0,0.0
5,6,Jake Ferguson (DAL),9.9,26,3,31.9,10.6,5.3,7.7,53.0,0.7,0.0,0.0,0.0,0.0
6,7,Mike Gesicki (CIN),11.1,37,3,29.6,9.9,4.7,6.0,52.0,0.3,0.0,0.0,0.0,0.0
7,8,George Kittle (SF),10.5,26,2,28.6,14.3,5.5,6.5,58.0,1.0,0.5,0.0,0.0,0.0
8,9,Hunter Henry (NE),11.3,35,3,25.6,8.5,4.0,6.0,45.3,0.3,0.0,0.0,0.0,0.0
9,10,Zach Ertz (WAS),10.7,21,3,24.8,8.3,4.0,4.3,42.7,0.7,0.0,0.0,0.0,0.0


### Calculate the correlations for the final stats across different conditions

In [26]:
# Define columns and columns to exclude for correlation calculation, excluding FPTS and FPTS/G
exclude_corr = ['FPTS/G', 'FPTS', 'G']
corr_columns = [col for col in final_columns if col not in exclude_corr]

# Define a function to calculate correlations
def compute_correlations(dataframe, corr_columns):
    return dataframe[corr_columns].corrwith(dataframe['FPTS/G'])

# Compute correlations for various conditions
corr_all = compute_correlations(df, corr_columns)
corr_nonzero = compute_correlations(df[df['FPTS/G'] > 0], corr_columns)
corr_top50 = compute_correlations(df[df['Rank'] <= 50], corr_columns)
corr_top25 = compute_correlations(df[df['Rank'] <= 25], corr_columns)

# Compile all correlations into a DataFrame for comparison
df_corr = pd.DataFrame({
    'All Players': corr_all,
    'FPTS > 0': corr_nonzero,
    'Top 50 Players': corr_top50,
    'Top 25 Players': corr_top25
})

# Calculate the average correlation across the four conditions, adding 'Average' column to DataFrame
df_corr['Correlation'] = df_corr.mean(axis=1)

# Display the correlation DataFrame
df_corr.round(2)

/home/codespace/.local/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/codespace/.local/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/codespace/.local/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/codespace/.local/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/codespace/.local/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/codespace/.local/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/hom

,All Players,FPTS > 0,Top 50 Players,Top 25 Players,Correlation
Y/R,0.66,0.21,0.12,0.25,0.31
LG,0.84,0.68,0.60,0.56,0.67
REC/game,0.97,0.96,0.93,0.92,0.94
TGT/game,0.93,0.90,0.88,0.85,0.89
YDS/game,0.96,0.95,0.93,0.91,0.94
20+/game,0.71,0.69,0.68,0.63,0.68
TD/game,0.46,0.41,0.27,0.09,0.31
ATT/game,0.03,-0.01,-0.10,-0.21,-0.07
YDS.1/game,0.03,-0.01,-0.08,-0.21,-0.07
TD.1/game,NaN,NaN,NaN,NaN,NaN


### Assign the weights for the final stats

In [27]:
# Calculate R^2 for the 'Average' correlation, adding 'R^2' column to DataFrame
df_corr['R^2'] = df_corr['Correlation'] ** 2

# Select stats with R^2 above a threshold for higher weight
high_weight_threshold = 0.5
specific_stats = df_corr[df_corr['R^2'] > high_weight_threshold].index.tolist()

# Select stats with R^2 below a threshold to exclude from final score
exclude_threshold = 0.1
exclude_stats = df_corr[df_corr['R^2'] < exclude_threshold].index.tolist()

# Define the calculation to assign weights
def weight_calc(row, specific_stats):
    if row.name in specific_stats:
        return 1 + row['R^2'] * 2 # Assign higher weight to specific stats
    else:
        return 1 + row['R^2'] # Assign weight to all other stats
    
# Assign weights based on the given criteria, adding 'Weight' column to DataFrame
df_corr['Weight'] = df_corr.apply(weight_calc, specific_stats=specific_stats, axis=1)

# Normalize weights
df_corr['Weight'] = df_corr['Weight'] / df_corr['Weight'].sum()

# Display the new columns in the DataFrame
df_corr[['Correlation', 'R^2', 'Weight']].round(2)

,Correlation,R^2,Weight
Y/R,0.31,0.10,0.07
LG,0.67,0.45,0.10
REC/game,0.94,0.89,0.18
TGT/game,0.89,0.79,0.17
YDS/game,0.94,0.88,0.18
20+/game,0.68,0.46,0.10
TD/game,0.31,0.09,0.07
ATT/game,-0.07,0.00,0.07
YDS.1/game,-0.07,0.00,0.07
TD.1/game,NaN,NaN,NaN


### Multiply the assigned weights to the final stats and calculate the score

In [28]:
# Multiply each relevant column by its corresponding weight
for col in corr_columns:
    if col not in exclude_stats:
        weight = df_corr.loc[col, 'Weight']
        df[col + '_weighted'] = (df[col] * weight).round(1)

# Extract the weighted columns
weight_columns = [col + '_weighted' for col in corr_columns if col not in exclude_stats]

# Display the new weighted stats columns
df_weight = df[['Rank', 'Player', 'FPTS/G'] + weight_columns]
df_weight.sort_values(by=['Rank'], ascending=True).head(10)

# Define select columns to be used for the average weighted score
avg = weight_columns + (['FPTS/G'] * 2)  # Giving higher weight to FPTS/G

# Calculate the average weighted score for the select columns
df['Score'] = df[avg].mean(axis=1).round(2)

# Normalize the scores to be out of 10
scaler = MinMaxScaler(feature_range=(0, 10))
df['Score'] = scaler.fit_transform(df[['Score']])

df_weight.head(10)

,Rank,Player,FPTS/G,LG_weighted,REC/game_weighted,TGT/game_weighted,YDS/game_weighted,20+/game_weighted,TD.1/game_weighted
0,1,Dallas Goedert (PHI),13.6,5.8,1.0,1.1,14.4,0.2,NaN
1,2,Brock Bowers (LV),12.6,2.6,1.1,1.2,11.9,0.1,NaN
2,3,Cole Kmet (CHI),11.3,1.8,0.9,1.0,7.7,0.0,NaN
3,4,Trey McBride (ARI),10.7,2.0,0.9,1.2,7.4,0.0,NaN
4,5,Isaiah Likely (BAL),10.7,4.7,0.7,0.9,8.5,0.0,NaN
5,6,Jake Ferguson (DAL),10.6,2.5,1.0,1.3,9.6,0.1,NaN
6,7,Mike Gesicki (CIN),9.9,3.5,0.9,1.0,9.4,0.0,NaN
7,8,George Kittle (SF),14.3,2.5,1.0,1.1,10.5,0.1,NaN
8,9,Hunter Henry (NE),8.5,3.3,0.7,1.0,8.2,0.0,NaN
9,10,Zach Ertz (WAS),8.3,2.0,0.7,0.7,7.7,0.1,NaN


### Model training

In [29]:
# Prepare features and target for model training
X = df[weight_columns]
y = df['Score']

# Handle missing values by imputing with mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

# Initialize Ridge regression model
ridge_model = Ridge()

# Cross-validation to evaluate the model
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validation MSE: {-cv_scores.mean()}')

# Train the model
ridge_model.fit(X_train, y_train)

# Predict and evaluate on the test set
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test Set MSE: {mse}')

# Incorporate Random Forest as an ensemble method
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest Test Set MSE: {rf_mse}')

# Average predictions from both models for final score
final_pred = (y_pred + rf_pred) / 2

# Create a DataFrame for the test set predictions
test_results = pd.DataFrame({'Final_Score': final_pred}, index=test_idx)

# Merge the test set predictions back into the original DataFrame
df = df.merge(test_results, how='left', left_index=True, right_index=True)

# Fill NaN values in 'Final_Score' column with the original 'Score' to handle missing indices
df['Final_Score'].fillna(df['Score'], inplace=True)

# Normalize final scores to be out of 10
df['Final_Score'] = scaler.fit_transform(df[['Final_Score']]).round(2)

# Rank the final scores
df['Final Rank'] = df['Final_Score'].rank(method='first', ascending=False).astype(int)

# Calculate the variance in ranking both ranks
df['Variance'] = df['Rank'] - df['Final Rank']

/home/codespace/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['TD.1/game_weighted']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


Cross-Validation MSE: 0.4186992208827289
Test Set MSE: 0.41761110238608184
Random Forest Test Set MSE: 0.09284178786581589


/tmp/ipykernel_3183/591390400.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Final_Score'].fillna(df['Score'], inplace=True)


### Display final results and export to Excel

In [30]:
# List of columns to be excluded
final_columns_exclude = ['Y/R', 'LG', 'ATT/game', 'YDS.1/game', 'TD.1/game']

# Exclude the specified columns from final_columns
final_columns = [col for col in final_columns if col not in final_columns_exclude]

# Create final analysis columns
analysis = df[['Rank', 'Final Rank', 'Player', 'Final_Score', 'Variance'] + final_columns]
analysis.set_index('Rank', inplace=True)
analysis = analysis.sort_values(by='Final Rank', ascending=True)

# Export to Excel
analysis.to_excel("TE_Analysis.xlsx", index=False)

# Display the top 30 rows
analysis.head(30)

,Final Rank,Player,Final_Score,Variance,G,FPTS,FPTS/G,REC/game,TGT/game,YDS/game,20+/game,TD/game
Rank,,,,,,,,,,,,
1,1,Dallas Goedert (PHI),10.00,0,3,40.9,13.6,5.7,6.7,79.7,1.7,0.0
8,2,George Kittle (SF),8.85,6,2,28.6,14.3,5.5,6.5,58.0,1.0,0.5
2,3,Brock Bowers (LV),8.51,-1,3,37.7,12.6,6.0,7.0,65.7,1.3,0.0
5,4,Isaiah Likely (BAL),7.35,1,3,32.1,10.7,4.0,5.3,47.0,0.3,0.3
6,5,Jake Ferguson (DAL),7.19,1,3,31.9,10.6,5.3,7.7,53.0,0.7,0.0
7,6,Mike Gesicki (CIN),7.04,1,3,29.6,9.9,4.7,6.0,52.0,0.3,0.0
3,7,Cole Kmet (CHI),6.93,-4,3,33.8,11.3,5.0,5.7,42.7,0.0,0.3
4,8,Trey McBride (ARI),6.71,-4,3,32.2,10.7,4.7,7.0,40.7,0.3,0.0
9,9,Hunter Henry (NE),6.18,0,3,25.6,8.5,4.0,6.0,45.3,0.3,0.0
